In [1]:
from operation import Operation
import numpy as np

matrix_x = np.array([[1, 2, 3], [np.nan, 5, np.nan], [7, 8, 9], [10, 11, 12]])
res = Operation.rolling_mean(matrix_x, 2)
res

array([[ 1. ,  2. ,  3. ],
       [ 1. ,  3.5,  3. ],
       [ 7. ,  6.5,  9. ],
       [ 8.5,  9.5, 10.5]])